## Using the RNC to look at preps in certain windows of verbs

In [1]:
pwd

'/Users/paigelee/Desktop/spring2021/clancy/verbhistograms'

In [2]:
import random
import string

In [3]:
from corus import load_morphoru_rnc

path = 'RNCgoldInUD_Morpho.conll'
records = load_morphoru_rnc(path)
rnccorpus = []
for record in records:
    rnccorpus.append(record)
len(rnccorpus)

98892

### looking at collocations around verbs

In [4]:
random.choice(random.choice(rnccorpus).tokens)

MorphoToken(
    text='.',
    lemma='.',
    pos='PUNCT',
    feats={},
    feats2={}
)

### All cxx (not just verbs)

In [5]:
def build_sentence(tokens):
    sentence = ''
    for i, token in enumerate(tokens):
        text = token.text
        space = ' '
        if token.pos == 'PUNCT':
            space = ''
        if text == None:
            text = ''
        sentence = sentence + space + text
    return sentence.strip()

# sample = random.sample(rnccorpus,1000)
sample = rnccorpus
windowDict = dict()
for window in range(2, 7):
    print('parsing',window,'...')
#     windowDict[window]= dict()
    for sent in sample:
        currentindex = -1
        for token in sent.tokens:
            currentindex += 1
            lemma = token.lemma
            nonalphasent = False
            propernounfound = False
            windowstart = currentindex
            # check window
            windowend = currentindex + window
            if windowend < len(sent.tokens):
                # get rid of proper nouns
                try:
                    for windowtok in sent.tokens[windowstart:windowend]:
                        if windowtok.feats2['NameType'] in ['Giv','Sur']:
                            propernounfound = True
                    if propernounfound:
                        break
                except:
                    propernounfound = False
                thissent = build_sentence(sent.tokens[windowstart:windowend])
                cxkey = thissent.lower()
                if '/' in cxkey or cxkey[-1] == ',' or cxkey[-2:] == '--' or cxkey[:2] == '--':
                    break
                alphacount = 0
                punctorspacecount = 0
                uppercount = 0
                for char in thissent:
                    if char.isnumeric():
                        nonalphasent = True
                    if char.isalpha():
                        alphacount += 1
                    if char in string.punctuation or char == ' ':
                        punctorspacecount += 1
                    if char.isupper():
                        uppercount += 1
                if uppercount == alphacount:
                    break
                if nonalphasent:
                    break
                if alphacount == 1:
                    break
                if punctorspacecount == len(thissent):
                    break
                windowDict.setdefault(cxkey, dict())
                windowDict[cxkey].setdefault('counts',0)
                windowDict[cxkey].setdefault('sents', [])
                windowDict[cxkey]['counts'] += 1
                windowDict[cxkey]['sents'].append(build_sentence(sent.tokens))

parsing 2 ...
parsing 3 ...
parsing 4 ...
parsing 5 ...
parsing 6 ...


In [6]:
# countdict = dict()
# for window in windowDict:
#     countdict[window] = []
#     for cxkey in windowDict[window]:
#         count = windowDict[window][cxkey]['counts']
#         countdict[window].append((count, cxkey))
# print('construction counts:')
# for wind in countdict:
#     print(wind, len(countdict[wind]))
# print()
# sortedcountdict = dict()
# for window in countdict:
#     sortedcountlist = sorted(countdict[window], reverse=True)
#     sortedcountdict[window] = sortedcountlist
# print('keys are...')
# sortedcountdict.keys()

In [8]:
cxx = list(windowDict.keys())
print(len(cxx))
print(random.sample(cxx, 5))

1130920
['латиноамериканской породы каудильо', 'всё чаще приходит', 'рынке интернет-рекламы доминируют', 'на другой день', 'вернуть хотя бы малые']


In [9]:
cxlist = []
for cx in windowDict:
    count = windowDict[cx]['counts']
    if count == 1:
        continue
    else:
        cxlist.append((count, cx))

In [10]:
len(cxlist)

41280

In [12]:
sortedcxlist = sorted(cxlist, reverse=True)

In [5]:
sortedcxlist[:100]

NameError: name 'sortedcxlist' is not defined

### Plotting constructions onto pca graph

### Cxx beginning with verbs only

In [87]:
def build_sentence(tokens):
    sentence = ''
    for i, token in enumerate(tokens):
        text = token.text
        space = ' '
        if token.pos == 'PUNCT':
            space = ''
        if text == None:
            text = ''
        sentence = sentence + space + text
    return sentence.strip()

# sample = random.sample(rnccorpus,1000)
sample = rnccorpus
windowDict = dict()
for window in range(2, 7):
    print('parsing',window,'...')
    windowDict[window]= dict()
    for sent in sample:
        currentindex = -1
        for token in sent.tokens:
            currentindex += 1
            if token.pos == 'VERB':
                verblemma = token.lemma
                windowstart = currentindex
                # check window
                windowend = currentindex + window
                if windowend < len(sent.tokens):
                    thissent = build_sentence(sent.tokens[windowstart:windowend])
                    cxkey = thissent.lower()
                    if '/' in cxkey or cxkey[-1] == ',' or cxkey[-2:] == '--':
                        break
                    windowDict[window].setdefault(cxkey, dict())
                    windowDict[window][cxkey].setdefault('counts',0)
                    windowDict[window][cxkey].setdefault('sents', [])
                    windowDict[window][cxkey]['counts'] += 1
                    windowDict[window][cxkey]['sents'].append(build_sentence(sent.tokens))

parsing 2 ...
parsing 3 ...
parsing 4 ...
parsing 5 ...
parsing 6 ...


In [88]:
countlist = []
for window in windowDict:
    for cxkey in windowDict[window]:
        count = windowDict[window][cxkey]['counts']
        countlist.append((count, cxkey))

In [95]:
len(countlist)

328466

In [89]:
sortedcountlist = sorted(countlist, reverse=True)

In [94]:
sortedcountlist[:100]

[(627, 'может быть'),
 (145, 'было бы'),
 (135, 'е.'),
 (85, 'сказать, что'),
 (81, 'могут быть'),
 (80, 'говорить о'),
 (68, 'сказал, что'),
 (65, 'см.'),
 (65, 'думаю, что'),
 (62, 'хотелось бы'),
 (59, 'хотел бы'),
 (59, 'см.:'),
 (57, 'был в'),
 (56, 'было в'),
 (54, 'сказал он'),
 (53, 'кажется, что'),
 (51, 'зависит от'),
 (51, 'есть и'),
 (49, 'знаю, что'),
 (49, 'есть у'),
 (49, 'есть в'),
 (48, 'состоит в'),
 (48, 'представляет собой'),
 (47, 'хочу сказать'),
 (47, 'могу сказать'),
 (43, 'идёт о'),
 (41, 'считаю, что'),
 (41, 'понял, что'),
 (41, 'есть ли'),
 (40, 'оказалось, что'),
 (40, 'могли бы'),
 (39, 'казалось, что'),
 (39, 'заключается в'),
 (39, 'говорим о'),
 (39, 'выяснилось, что'),
 (39, 'было не'),
 (38, 'смотрит на'),
 (38, 'была в'),
 (37, 'сказал я'),
 (37, 'начиная с'),
 (37, 'имею в'),
 (37, 'говорят, что'),
 (36, 'относится к'),
 (36, 'было и'),
 (35, 'считают, что'),
 (35, 'казалось бы'),
 (35, 'имею в виду'),
 (35, 'был бы'),
 (34, 'спросил я'),
 (34, 'сос

## prep + case!!!

In [56]:
def build_sentence(tokens, currentindex, adpindex, caseindex):
    sentence = ''
    for i, token in enumerate(tokens):
        if token.pos == 'PUNCT':
            space = ''
        else:
            space = ' '
        if i in [currentindex, adpindex, caseindex]:
            text = token.text.upper()
        else:
            text = token.text
        if text == None:
            text = ''
        sentence = sentence + space + text
    return sentence.strip()

# sample = random.sample(rnccorpus,100)
sample = rnccorpus
windowList = []
for window in range(-3, 4):
    dataDict = dict()
    for sent in sample:
        currentindex = 0
        for token in sent.tokens:
            if token.pos == 'VERB':
                verblemma = token.lemma
#                 if verblemma not in okayverbs:
#                     break
                if verblemma not in dataDict:
                    dataDict[verblemma] = dict()
                    dataDict[verblemma]['counts'] = dict()
                    dataDict[verblemma]['sentences'] = dict()
                adpindex = currentindex + window
                if adpindex >= 0 and adpindex < len(sent.tokens):
                    adptoken = sent.tokens[adpindex]
                    if adptoken.pos == 'PUNCT' and adpindex + 1 >= 0 and adpindex + 1 < len(sent.tokens):
                        adpindex = adpindex + 1
                    if adptoken.pos == 'ADP':
                        adplemma = adptoken.lemma
                        caseindex = adpindex + 1
    
                        if caseindex >= 0 and caseindex < len(sent.tokens):
                            casetoken = sent.tokens[caseindex]
                            if casetoken.pos == 'PUNCT' and caseindex + 1 >= 0 and caseindex + 1 < len(sent.tokens):
                                caseindex = caseindex + 1
                            casetoken = sent.tokens[caseindex]
                            try:
                                case = casetoken.feats['Case']
                                if case == 'Ins':
                                    case = 'Inst'
                                adpandcase = adplemma + ' + ' + case.upper()
                            except:
                                break
                                
                    else:
                        if window in [-1,1]:
                            # not an adposition directly before or after verb
                            try: 
                                if adpindex - 1 >= 0:
                                    # need to make sure prev token is not adp
                                    if sent.tokens[adpindex - 1].pos != 'ADP':
                                        case = adptoken.feats['Case']
                                        if case == 'Ins':
                                            case = 'Inst'
                                        adpandcase = 'No prep + ' + case.upper()
                                        caseindex = currentindex
                            except:
                                break
                        else:
                            break
                            
                    if adpandcase not in dataDict[verblemma]['counts']:                               
                        dataDict[verblemma]['counts'][adpandcase] = 0
                        dataDict[verblemma]['sentences'][adpandcase] = []

                    dataDict[verblemma]['counts'][adpandcase] += 1
                    formattedsent = build_sentence(sent.tokens, currentindex, adpindex, caseindex)
                    if formattedsent not in dataDict[verblemma]['sentences'][adpandcase]:
                        dataDict[verblemma]['sentences'][adpandcase].append(formattedsent)
                                
                    
            currentindex += 1
    print(f'window size of {window} complete...')
    windowList.append(dataDict)

window size of -3 complete...
window size of -2 complete...
window size of -1 complete...
window size of 0 complete...
window size of 1 complete...
window size of 2 complete...
window size of 3 complete...


In [57]:
windowList[2]['сходить']['sentences']['No prep + ACC']

['ОБЯЗАТЕЛЬНО СХОЖУ.',
 'А когда первый РАЗ СХОДИЛА в кино и увидела музыкантов, которые играют в фойе, мне стало жаль моего бедного дедушку: зрители переговаривались, жевали бутерброды, шуршали газетами, а старые люди на сцене играли вальс.',
 'В ПЛАЦКАРТУ СХОДИ, там быстро развеешься.',
 'Наш корреспондент Микаэль Укин получил ЗАДАНИЕ СХОДИТЬ в Мавзолей.',
 'Там учителя с УМА СХОДЯТ, не ПОЙМУТ, откуда такое взялось.',
 'Я ему полис ОТДАЛА/ пусть там на РАБОТЕ СХОДИТ...',
 'СЕЙЧАС СХОЖУ/ ГОВОРИТ/ завтра/ говорит/ этот/ паспорт заберу...',
 'Я этот рубль БЕРЕГУ/ то в КИНО СХОДИТЬ/ то в кафе сходить/ куда-нибудь.',
 'Я этот рубль БЕРЕГУ/ то в кино сходить/ то в КАФЕ СХОДИТЬ/ куда-нибудь.']

In [45]:
# removing uncommon verbs (<10 occurrences)
uncommonVerbs = set()
totalcount = dict()
for windowDict in windowList:
    for verb in windowDict:
        if verb not in totalcount:
            totalcount[verb] = 0
        countDict = windowDict[verb]['counts']
        for prep in windowDict[verb]['counts']:
            totalcount[verb] += windowDict[verb]['counts'][prep]
for verb in totalcount:
    if totalcount[verb] < 30:
        uncommonVerbs.add(verb)
print(len(uncommonVerbs))

92


In [7]:
for verbtoremove in uncommonVerbs:
    for windowDict in windowList:
        try:
            del windowDict[verbtoremove]
        except:
            False

In [8]:
len(windowList[1])

597

In [9]:
windowList[1]['читать']['counts']

{'в + LOC': 4,
 'среди + GEN': 1,
 'на + LOC': 1,
 'с + INST': 2,
 'из + GEN': 1,
 'после + GEN': 1,
 'по + DAT': 1}

In [12]:
cd verbhistograms/

/Users/paigelee/Desktop/spring2021/clancy/verbhistograms


## for csv

In [13]:
allcxlabels = set()
for windowDict in windowList:
    for verb in windowDict:
        countdict = windowDict[verb]['counts']
        for cxlabel in countdict:
            allcxlabels.add(cxlabel)
allcxlabels = list(allcxlabels)

In [14]:
# getting rid of cxx with count < 10
totalcxcounts = dict()
for windowDict in windowList:
    for verb in windowDict:
        countdict = windowDict[verb]['counts']
        for cx in countdict:
            if cx not in totalcxcounts:
                totalcxcounts[cx] = 0
            totalcxcounts[cx] += 1
lst = []
for cx in totalcxcounts:
    lst.append((totalcxcounts[cx],cx))
i = 0
greaterthan10cx = []
for ct, cx in lst:
    if ct > 10:
        greaterthan10cx.append(cx)

In [15]:
# verb, windowsize, prep1, prep2, prep3, prep4, prep5...
rows = []
# cxused = allcxlabels
cxused = greaterthan10cx
for windowIndex, windowDict in enumerate(windowList):
    windowSize = windowIndex - 3
    for verb in windowDict:
        countdict = windowDict[verb]['counts']
        csvline = [verb, windowSize]
        for cxlabel in cxused:
            if cxlabel in countdict:
                count = countdict[cxlabel]
            else:
                count = 0
            csvline.append(count)
        rows.append(csvline)

In [18]:
r = random.choice(rows)
print(r)

['встретиться', -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 2, 2, 0, 1]


In [19]:
fields = ['Verb','WindowSize']

for cxlabel in greaterthan10cx:
    fields.append(cxlabel)

In [20]:
pwd

'/Users/paigelee/Desktop/spring2021/clancy/verbhistograms'

In [21]:
import csv
# writing to csv file 
filename = 'csvdata.csv'
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [33]:
cd verbhistograms/

/Users/paigelee/Desktop/spring2021/clancy/verbhistograms


In [22]:
sentenceDict = dict()
windowSize = -3
for windowDict in windowList:
    sentenceDict[windowSize] = dict()
    for verb in windowDict:
        sentenceDict[windowSize][verb] = dict()
        for cx in windowDict[verb]['sentences']:
            sentenceDict[windowSize][verb][cx] = windowDict[verb]['sentences'][cx][:10]
    windowSize += 1

In [23]:
totalsents = 0
for windowDict in windowList:
    for verb in windowDict:
        for prep in windowDict[verb]['sentences']:
            totalsents += len(windowDict[verb]['sentences'][prep])
print(totalsents)

82232


In [24]:
verbDict = dict()

for verb in windowList[0]:
    verbDict[verb] = dict()
windowSize = -3
for windowDict in windowList:
    for verb in windowDict:
        verbDict[verb][windowSize] = dict()
        for cx in windowDict[verb]['sentences']:
            verbDict[verb][windowSize][cx] = list()
            for sent in windowDict[verb]['sentences'][cx]:
                verbDict[verb][windowSize][cx].append(sent)
    windowSize += 1

In [37]:
verbDict['помочь'][1].keys()

dict_keys(['No prep + DAT', 'No prep + ACC', 'No prep + NOM', 'No prep + INST'])

In [34]:
pwd

'/Users/paigelee/Desktop/spring2021/clancy/verbhistograms'

In [38]:
import json
for verb in verbDict:
    with open(f'data/{verb}.json', 'w', encoding='utf8') as jsonfile:
        json.dump(verbDict[verb], jsonfile, ensure_ascii=False)

### write sentence dictionary to json

In [78]:
# writing dictionary to json

import json

filepath = 'sentences.json'

with open(filepath, 'w', encoding='utf8') as json_file:
    json.dump(windowList, json_file, ensure_ascii=False)
    
print(filepath,'written successfully.')

sentences.json written successfully.
